In [1]:
%env THEANO_FLAGS="device=gpu4"

env: THEANO_FLAGS="device=gpu4"


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook as tqdm
from PIL import Image

from librosa import load, logamplitude
from librosa.feature import melspectrogram

In [3]:
SOUND_SHAPE = (500, 513)

In [ ]:
df = pd.DataFrame.from_csv("pronuns.csv")

In [4]:
def get_spectrogram(img_name):
    return np.array(Image.open("../../data/images/{}.png".format(img_name)))

# def get_spectrogram(path):
#     """Строим спектограмму из wav файла"""
#     y, sr = load("mongodb/sounds/%s.wav" % path)
#     S = melspectrogram(y, sr=sr, n_mels=100)
#     log_S = logamplitude(S, ref_power=np.max)
#     return log_S

In [ ]:
plt.imshow(get_spectrogram("585fda10698f828c848d862d"))
get_spectrogram("585fda10698f828c848d862d").shape

In [ ]:
np.min(get_spectrogram("585fda10698f828c848d862d"))

In [ ]:
df.head()

In [ ]:
for _id in tqdm(set([i[:-4] for i in os.listdir("mongodb/sounds")])):
    get_spectrogram("../../sounds/%s.wav" % _id)

In [ ]:
final_train = {}
for user in tqdm(df.user.unique()):
    user_sp = {}
    user_sp_data = df[df.user == user].pronun_id.values
    final_train[user] = user_sp_data

In [ ]:
from scipy.sparse import bsr_matrix as sp_matrix

In [5]:
def as_matrix(ar, shape):
    ret_mat = np.zeros((len(ar), shape[0], shape[1]), dtype="float16")
    for i, vec in enumerate(ar):
        try:
            if vec.shape[0] > shape[0]:
                ret_mat[i, :, :] = vec[:shape[0], :]/200.
            else:
                ret_mat[i, :vec.shape[0], :] = vec/200.
        except IndexError:
            print(vec.shape)
            raise IndexError

    return ret_mat

In [6]:
import dill
from numpy.random import choice

class VoicesData:
    def __init__(self, path='users.dl'):
        # Путь до dill моделf
        self.path = path
#         Defaultdict информации
        self.base = dill.load(open(self.path, 'rb'))
#         self.base = final_train
    def __getitem__(self, item):
        return self.base[item]
    def save(self):
        """Поскольку это не стандартная DB, тут нужна функция сохранения"""
        dill.dump(self.base, open(self.path, 'wb'))

    def get_train_vec(self, shape=SOUND_SHAPE):
        """Делаем из данных train выборку"""
        for key in tqdm(self.base.keys()): 
            train = []
            if len(self.base[key]) >= 2:
                for _ in range(0, len(self.base[key]), 2):
                    values = choice(list(self.base[key]), 2)
                    a = np.asarray(get_spectrogram(values[0]))
                    b = np.asarray(get_spectrogram(values[1]))
                    other = choice(list(self.base), 1)[0]
                    c = np.asarray(get_spectrogram(choice(list(self.base[other]), 1)[0]))

                    other_value = c
                    value_first = a
                    value_second = b
#                     print(a.shape)
                    # other_value.resize(shape) # value_first.resize(shape) # value_second.resize(shape)
                    some = as_matrix((value_first, other_value, value_second), shape)
                    train.append(some)
#                     break
                    
#                 print(len(train))
                yield train
            else:
#                 print("I'm empty")
                pass

    def get_train_people(self, shape=SOUND_SHAPE, count=10000):
        X = []
        y = []
        people = list(self.base.keys())
        for _ in tqdm(range(0, count//2)):
            man = choice(people, 1)
            track = choice(list(self.base[man[0]]), 2)

            a = get_spectrogram(track[0])
            b = get_spectrogram(track[1])

            X.extend([as_matrix([a, b], shape)])

        y.extend([1 for _ in range(len(X))])
        X_ol = len(X)
#                 break

        
        for _ in tqdm(range(count//2, count)):
            man1 = choice(people, 1)[0]
            people_new = people[:]
            people_new.remove(man1)

            man2 = choice(people_new, 1)[0]
            
#             print(man1)
            sou = self.base[man1]
            a = choice(list(sou), 1)[0]
            sou = self.base[man2]
            b = choice(list(sou), 1)[0]
            
            X.append(as_matrix([get_spectrogram(a),
                      get_spectrogram(b)], shape))
        y.extend([0 for _ in range(len(X) - X_ol)])
        return np.array(X, dtype="float16"), np.array(y)

In [ ]:
choice([0], 2)

In [7]:
users = VoicesData("../../data/users.dl")

In [ ]:
users.save()

In [18]:
X, y = users.get_train_people(count=40000)

In [ ]:
data = users.get_train_vec()

In [ ]:
del users

In [ ]:
np.save("X", X)
np.save("y", y)

In [8]:
import theano
import theano.tensor as T

import lasagne

from lasagne.layers import InputLayer, DenseLayer, ReshapeLayer, Conv1DLayer, MaxPool1DLayer, GlobalPoolLayer, \
    get_output, get_all_params, get_all_param_values, set_all_param_values

from lasagne.nonlinearities import very_leaky_rectify, tanh

from lasagne.updates import adagrad

def make_speechtovec(incoming, sound_shape, num_units, **kwargs):
    """
    :param incoming: the layer feeding into this layer, or the expected input shape.
    :param sound_shape: shape of freq x time
    :param num_units: output vector dimension
    """


    input_reshape = ReshapeLayer(incoming, (-1,) + sound_shape)  # Сворачиваем все записи друг за другом
    convolution = Conv1DLayer(input_reshape, num_filters=100, filter_size=5,
                              nonlinearity=very_leaky_rectify, name="Convolutional")
    pooling = MaxPool1DLayer(convolution, 2)
    global_pooling = GlobalPoolLayer(pooling)
    dense = DenseLayer(global_pooling, num_units=300, name="Dense")
    output_dense = DenseLayer(dense, num_units=num_units, nonlinearity=lasagne.nonlinearities.linear, name='output')
    all_vectors_output = ReshapeLayer(output_dense, (-1, 3, num_units))

    return all_vectors_output, output_dense

Using gpu device 4: GeForce GTX 1080 (CNMeM is enabled with initial size: 45.0% of memory, cuDNN 5105)


In [9]:
input_triplets = T.tensor4("Triplets input", dtype="float32")
target = T.ivector("Target")

In [ ]:
triplets_input = InputLayer((None, 3) + SOUND_SHAPE, input_var=input_triplets)
# people_inputs = InputLayer((None, 2, 500, 513))
# from lasagne.layers import dimshuffle
# dimshuffle(triplets_input,[0,1,3,2])
vectorizer, _ = make_speechtovec(lasagne.layers.dimshuffle(triplets_input,[0,1,3,2]), SOUND_SHAPE[::-1], 300)

In [ ]:
lasagne.layers.set_all_param_values(vectorizer, param)

In [ ]:
all_pred = get_output(vectorizer)
params = get_all_params(vectorizer, trainable=True)

In [ ]:
def loss_func(all_predicted):
    def distance_sq(x1, x2):
        return T.sum(T.sqr(x1 - x2))

    d1 = distance_sq(all_predicted[:, 0], all_predicted[:, 1])
    d2 = distance_sq(all_predicted[:, 0], all_predicted[:, 2])
    alpha = 1e-2

    return T.maximum(d1 + alpha, 0) - T.maximum(d2 + alpha, 0)

In [ ]:
def cos_sim(vec1, vec2):
    numerator = T.sum(vec1*vec2)
    denumenator = T.sqrt(T.sum(vec1**2)*T.sum(vec2**2))
    return numerator/denumenator

def denum_fun(v1, v2, similar_v, epsilon=0.0001):
    if similar_v:
        return (cos_sim(v1, v2) + 1)/2 + epsilon
    else:
        return (cos_sim(v1, v2) - 1)/2 + epsilon

def loss_func_new(all_predicted, epsilon=0.0001):
    def distance_sq(x1, x2):
        return T.sum(T.sqr(x1 - x2))

    d1 = distance_sq(all_predicted[:, 0], all_predicted[:, 1])
    d2 = distance_sq(all_predicted[:, 0], all_predicted[:, 2])
    
    d1 /= denum_fun(all_predicted[:, 0], all_predicted[:, 1], True)
    d2 /= denum_fun(all_predicted[:, 0], all_predicted[:, 2], False)
    alpha = 1e-2

    return T.maximum(d1 + alpha, 0) - T.maximum(d2 + alpha, 0)

In [ ]:
loss = loss_func(all_pred)

updates = adagrad(loss, params)

In [ ]:
train = theano.function([triplets_input.input_var], updates=updates)

In [ ]:
shape = None
users = VoicesData("../../data/users.dl")
for i in range(5):
    data = users.get_train_vec()
    for t in data:
        try:
            t = np.array(t)

            if t.shape[0] > 1000:
                train(t[:500])
                train(t[500:1000])
                train(t[1000:])
            elif t.shape[0] > 500:
                train(t[:500])
                train(t[500:])
            else:
                train(t)

            shape = t.shape
            del t
        except MemoryError:
            print(t.shape, shape)
            break

In [ ]:
np.save("../../data/weights_vec_new_new.npy", lasagne.layers.get_all_param_values(vectorizer))

In [ ]:
param = np.load("../../data/weights_vec_new.npy")

In [10]:
param = np.load("../../data/tvorog_vectorizer.npy")

In [ ]:
pr = theano.function([triplets_input.input_var], all_pred)

In [ ]:
shape = None
for t in data:
    print(pr(t))
    break

In [ ]:
t

In [ ]:
np.array(t).dtype

In [ ]:
for l in get_all_layers(vectorizer):
    print (l, l.output_shape)

In [11]:
siminput = T.tensor3("Similar voice")

In [12]:
triplets_input = InputLayer((None, 2) + SOUND_SHAPE, input_var=input_triplets)
# nn = lasagne.layers.batch_norm(triplets_input)
# people_inputs = InputLayer((None, 2, 500, 513))
# from lasagne.layers import dimshuffle
_ ,vectorizer= make_speechtovec(lasagne.layers.dimshuffle(triplets_input,[0,1,3,2]), SOUND_SHAPE[::-1], 300)

similar_inp = lasagne.layers.InputLayer((None, 2, 300), input_var=siminput)
# vector_output = ReshapeLayer(vectorizer, (-1, 2, 300))
nn = lasagne.layers.batch_norm(similar_inp)
conv_layer = Conv1DLayer(nn, 100, 2)
nn = lasagne.layers.batch_norm(conv_layer)
dense0 = DenseLayer(nn, 150)
nn = lasagne.layers.batch_norm(dense0)
dense0 = DenseLayer(nn, 50)
nn = lasagne.layers.batch_norm(dense0)
output = DenseLayer(nn, 1, nonlinearity=lasagne.nonlinearities.sigmoid)
# dense0 = DenseLayer(vector_output, 100)
# nn = lasagne.layers.batch_norm(dense0)
# output = DenseLayer(nn, 1, nonlinearity=lasagne.nonlinearities.sigmoid)

In [13]:
lasagne.layers.set_all_param_values(vectorizer, param)

In [14]:
predict = lasagne.layers.get_output(output)
vec_pr = lasagne.layers.get_output(vectorizer)

In [ ]:
predict

In [15]:
parametrs = lasagne.layers.get_all_params(output, trainable=True)

In [ ]:
parametrs

In [16]:
loss = lasagne.objectives.binary_crossentropy(predict, target).sum()
acc = lasagne.objectives.binary_accuracy(predict, target).mean()
updates = lasagne.updates.adamax(loss, parametrs,learning_rate = 0.0001)

In [17]:
train = theano.function([similar_inp.input_var, target] ,updates=updates, allow_input_downcast=True)
vectoriz = theano.function([triplets_input.input_var], vec_pr, allow_input_downcast=True)
presd = theano.function([similar_inp.input_var],predict ,allow_input_downcast=True)

In [19]:
ind = np.arange(len(X))
np.random.shuffle(ind)


In [20]:
in_train, in_test = ind[1000:], ind[:1000]
# y_train, y_test = Y[1000:], Y[:1000]

In [21]:
def iterate_minibatches(inputs, targets, ind, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(ind))
        np.random.shuffle(indices)
    for start_idx in tqdm(range(0, len(ind) - batchsize + 1, batchsize)):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[ind[excerpt]], targets[ind[excerpt]]

In [22]:
from time import time

In [ ]:
lasagne.layers.set_all_param_values(output, np.load("../../data/weights/symvoice_weights.npy"))

In [ ]:
EPOCH = 100

for epoch in range(EPOCH):
    st = time()
    for i, batch in enumerate(iterate_minibatches(X, y, in_train, 1000)):
#         print("Hey")
        x_tr, y_tr = batch
        train(vectoriz(x_tr).reshape((-1, 2, 300)), y_tr)
#         print("Hop")
#         break
        if i > 100:
            break
#     break
    print('\r', "Time: ", (time()-st)/60.)
    print("\tAccuracy: ", roc_auc_score(y[in_test], presd(vectoriz(X[in_test]).reshape((-1, 2, 300)))))

In [ ]:
del train, vectoriz, presd

In [ ]:
lasagne.layers.get_all_param_values(vectorizer)

In [ ]:
acc_fun(X[in_test], y[in_test])

In [ ]:
y[in_test].sum()/len(y[in_test])

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y[in_test], presd(vectoriz(X[in_test]).reshape((-1, 2, 300))))

In [ ]:
np.save("../../data/simvoice_weights.npy", lasagne.layers.get_all_param_values(output))

In [ ]:
np.save("../../data/vectorizer_weights.npy", lasagne.layers.get_all_param_values(vectorizer))

In [ ]:
t = None

In [ ]:
vectoriz(X[in_test])